In [ ]:
# 01_data_preparation.ipynb
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import time
from tqdm import tqdm
from bertopic import BERTopic
import jieba  # For Chinese word segmentation
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import yaml
from itables import show

In [ ]:
# Get the directory of the current notebook
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Parent directory is the project root
PROJECT_ROOT = os.path.dirname(current_dir)
print(f"Project root identified as: {PROJECT_ROOT}")

# Set paths - CONFIGURABLE PARAMETERS
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
DATA_SUBFOLDER = "data2"  # Change this to the specific subfolder where your data is located
DATA_FILENAME = "data.csv"  # Change this to your data filename

# Construct full path to the data file
DATA_PATH = os.path.join(DATA_DIR, DATA_SUBFOLDER, DATA_FILENAME)
print(f"Looking for data at: {DATA_PATH}")

# Load the data
df = pd.read_csv(DATA_PATH)
print(f"Loaded dataset with shape: {df.shape}")

# ensure the text fields are strings
df['english_text'] = df['english_text'].astype(str)
df['chinese_text'] = df['chinese_text'].astype(str)
df['technique'] = df['technique'].astype(str)
df['intent'] = df['intent'].astype(str)


# Display basic information
df.info()
df.head(10)

In [ ]:
secrets_path = os.path.join(PROJECT_ROOT, "secrets.yaml")
with open(secrets_path, 'r') as f:
    secrets = yaml.safe_load(f)

if "gpt_key" not in secrets:
    raise ValueError("GPT key not found in secrets.yml")

In [ ]:
# Count non-null values per column
print("\nNon-null counts:")
print(df.count())

# Check for unique values in categorical columns
for col in ['success', 'technique', 'intent']:
    if df[col].dtype == 'object' or df[col].nunique() < 10:
        print(f"\nUnique values in {col}:")
        print(df[col].value_counts())

In [ ]:
# Text length distribution
df['english_text_length'] = df['english_text'].apply(lambda x: len(str(x)) if not pd.isna(x) else 0)
df['chinese_text_length'] = df['chinese_text'].apply(lambda x: len(str(x)) if not pd.isna(x) else 0)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df['english_text_length'])
plt.title('English Text Length Distribution')
plt.xlabel('Character Count')

plt.subplot(1, 2, 2)
sns.histplot(df['chinese_text_length'])
plt.title('Chinese Text Length Distribution')
plt.xlabel('Character Count')
plt.tight_layout()
plt.show()

In [ ]:
from deep_translator import GoogleTranslator

def translate_to_chinese(text):
    try:
        if text is None or text.strip() == '':
            return None
        translation = GoogleTranslator(source='en', target='zh-CN').translate(text)
        return translation
    except Exception as e:
        print(f"Error translating text: {text}, Error: {e}")
        return None

# Apply translation function to each row in the DataFrame
df["chinese_translation_google"] = df["english_text"].apply(translate_to_chinese)

show(df[["english_text", "chinese_translation_google", "chinese_text"]].head())

In [ ]:
def translate_to_english(text):
    try:
        if text is None or text.strip() == '':
            return None
        translation = GoogleTranslator(source='zh-CN', target='en').translate(text)
        return translation
    except Exception as e:
        print(f"Error translating text: {text}, Error: {e}")
        return None

# Apply translation function to each row in the DataFrame
df["english_translation_google"] = df["chinese_text"].apply(translate_to_english)

show(df[["chinese_text", "english_translation_google", "english_text"]].head())

In [ ]:
def generate_embeddings(df, text_columns=['english_text', 'chinese_text', 'english_translation_google'], model="text-embedding-3-small"):
    """Generate OpenAI embeddings for specified text columns.
    
    Args:
        df (pandas.DataFrame): DataFrame containing the text columns
        text_columns (list): List of column names to generate embeddings for
        model (str): OpenAI embedding model to use
        
    Returns:
        pandas.DataFrame: Original DataFrame with added embedding columns
    """
    # Initialize OpenAI client with API key from secrets
    client = OpenAI(api_key=secrets["gpt_key"])
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_with_embeddings = df.copy()
    
    # Track progress with tqdm
    total_embeddings = len(df) * len(text_columns)
    progress_bar = tqdm(total=total_embeddings, desc="Generating embeddings")
    
    # Generate embeddings for each specified column
    for col in text_columns:
        # Create a new column name for the embeddings
        embedding_col_name = f"{col}_embedding"
        
        # Initialize the embedding column with empty lists
        df_with_embeddings[embedding_col_name] = [[] for _ in range(len(df))]
        
        # Process each row
        for idx, row in df.iterrows():
            # Skip empty or NaN text
            if pd.isna(row[col]) or row[col] == "":
                progress_bar.update(1)
                continue
                
            try:
                # Convert to string and generate embedding
                text = str(row[col])
                response = client.embeddings.create(
                    input=text,
                    model=model
                )
                embedding = response.data[0].embedding
                
                # Store the embedding in the DataFrame
                df_with_embeddings.at[idx, embedding_col_name] = embedding
                
                # Rate limiting to avoid API throttling
                #time.sleep(0.01)  # Reduced sleep time for the smaller model
                
            except Exception as e:
                print(f"Error generating embedding for row {idx}, column {col}: {str(e)}")
            
            # Update progress bar
            progress_bar.update(1)
    
    progress_bar.close()
    return df_with_embeddings

# Generate embeddings using the specified model
print("Starting embedding generation...")
df_with_embeddings = generate_embeddings(df, model="text-embedding-3-small")
df = df_with_embeddings  # Update the main dataframe

In [ ]:
# Save the processed data with embeddings to the same subfolder as the original data
output_path = os.path.join(DATA_DIR, DATA_SUBFOLDER, "processed_data.pkl")

# Save as pickle for an efficient binary format that preserves all Python objects exactly as they are
df.to_pickle(output_path)
print(f"Saved processed data with embeddings to: {output_path}")

print("Data preparation complete!")

# Verification of Final Data Frame

In [ ]:
# Print a random vector embedding sample from English and Chinese
import random

# Check if we have any English embeddings
english_embedding_col = 'english_text_embedding'
if english_embedding_col in df.columns and len(df) > 0:
    # Get non-empty English embeddings
    non_empty_english = df[df[english_embedding_col].apply(lambda x: isinstance(x, list) and len(x) > 0)]
    if len(non_empty_english) > 0:
        # Select a random row
        random_row = random.choice(range(len(non_empty_english)))
        random_idx = non_empty_english.index[random_row]
        
        # Get the embedding vector
        english_embedding = df.at[random_idx, english_embedding_col]
        
        # Print some sample info
        print("\nRandom English Embedding Sample:")
        print(f"Row index: {random_idx}")
        print(f"Vector length: {len(english_embedding)}")
        print(f"First 5 dimensions: {english_embedding[:5]}")
        print(f"Last 5 dimensions: {english_embedding[-5:]}")
    else:
        print("\nNo non-empty English embeddings found.")
else:
    print(f"\nNo '{english_embedding_col}' column found in the DataFrame.")

# Check if we have any Chinese embeddings
chinese_embedding_col = 'chinese_text_embedding'
if chinese_embedding_col in df.columns and len(df) > 0:
    # Get non-empty Chinese embeddings
    non_empty_chinese = df[df[chinese_embedding_col].apply(lambda x: isinstance(x, list) and len(x) > 0)]
    if len(non_empty_chinese) > 0:
        # Select a random row
        random_row = random.choice(range(len(non_empty_chinese)))
        random_idx = non_empty_chinese.index[random_row]
        
        # Get the embedding vector
        chinese_embedding = df.at[random_idx, chinese_embedding_col]
        
        # Print some sample info
        print("\nRandom Chinese Embedding Sample:")
        print(f"Row index: {random_idx}")
        print(f"Vector length: {len(chinese_embedding)}")
        print(f"First 5 dimensions: {chinese_embedding[:5]}")
        print(f"Last 5 dimensions: {chinese_embedding[-5:]}")
    else:
        print("\nNo non-empty Chinese embeddings found.")
else:
    print(f"\nNo '{chinese_embedding_col}' column found in the DataFrame.")



In [ ]:
show(df)